범주형 변수를 처리하는 3가지 방법에 대해서 배운다

### **1.Drop Categorical Variables**
아예 범주형 데이터를 없앤다.  
단, 이 열의 데이터가 유용하지 않을 경우만 사용

### **2.Ordinal Encoding**
순서형 인코딩 : 매우좋음 5 좋음 4 보통 3 등과 같이 각 값들에 점수를 매긴다.  
단, 순서를 정할 수 있는 경우에만 사용

### **3.One-Hot Encoding**
원핫 인코딩 : 순서를 정할 수 없을 경우에 사용한다.  
  
<img src = "https://i.imgur.com/TW5m0aJ.png" height='200px' width = '700px'>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datafile/melb_data.csv")

y = data.Price
X = data.drop(['Price'], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X,y,train_size = 0.8,test_size=0.2, random_state=0)

cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()]

X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype=="object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()


,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


In [14]:
s = (X_train.dtypes == "object")
object_cols = list(s[s].index)

print(object_cols)

['Type', 'Method', 'Regionname']


* Categorical Variables 분류


In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(preds, y_valid)  


### **1번 방법**

In [18]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

175703.48185157913


### **2번 방법**

In [19]:
from sklearn.preprocessing import OrdinalEncoder

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))


165936.40548390493


### **3번 방법**

In [32]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

C:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


166089.4893009678


C:\Users\Administrator\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


* handle_unknown : error, ignore 두가지(default = error) 알수없는 값을 만났을 때 오류를 발생시킬지 무시할지
* sparse : True는 희소행렬 반환, False는 배열 반환 (default = True)